In [1]:
#!pip install playwright

In [2]:
#!playwright install

In [3]:
import warnings, re, json
warnings.filterwarnings("ignore")

import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from playwright.async_api import async_playwright


px.defaults.template = "plotly_white"
px.defaults.color_continuous_scale = "RdBu"

In [4]:
df = pd.read_csv("keepa_product_links.csv",encoding="utf-8-sig")
pd.set_option('display.max_columns', None)
df.head()

,Amazon: Lowest,Amazon: Highest,Amazon: Availability of the Amazon offer,Amazon: 30 days avg.,Amazon: 90 days OOS,Amazon: 90 days avg.,Amazon: 180 days avg.,New: 30 days drop %,New: 90 days drop %,New: Last visit,New: Current,New: 90 days avg.,New: 30 days avg.,New: 180 days avg.,New: Highest,New: 90 days OOS,New: Lowest,eBay New 🚚: 90 days avg.,eBay New 🚚: Current,eBay New 🚚: 90 days drop %,eBay New 🚚: Highest,eBay New 🚚: 180 days avg.,eBay New 🚚: 30 days drop %,eBay New 🚚: 30 days avg.,eBay New 🚚: Lowest,Reviews: Review Count - 30 days drop %,Reviews: Review Count - 30 days avg.,Reviews: Review Count - 180 days avg.,Reviews: Review Count,Reviews: Review Count - 90 days drop %,Reviews: Review Count - 90 days avg.,Reviews: Rating,Buy Box 🚚: 30 days avg.,Buy Box 🚚: Highest,Buy Box 🚚: 90 days drop %,Buy Box 🚚: 180 days avg.,Buy Box: Is FBA,Buy Box 🚚: 90 days OOS,Buy Box 🚚: 90 days avg.,Buy Box 🚚: Stock,Buy Box: Unqualified,Buy Box: Winner Count 180 days,Buy Box: % Top Seller 30 days,Buy Box: % Top Seller 365 days,Buy Box 🚚: Lowest,Buy Box 🚚: Last visit,Buy Box 🚚: 30 days drop %,Buy Box 🚚: Current,Buy Box: % Top Seller 180 days,Buy Box: % Top Seller 90 days,Prime Eligible (Buy Box),Sales Rank: Drops last 90 days,Sales Rank: 90 days avg.,Sales Rank: Drops last 180 days,Sales Rank: 30 days drop %,Sales Rank: 180 days avg.,Sales Rank: 30 days avg.,Sales Rank: Current,Sales Rank: Highest,Bought in past month,Sales Rank: Lowest,Sales Rank: Drops last 30 days,Sales Rank: 90 days drop %,"New, 3rd Party FBM 🚚: Highest","New, 3rd Party FBM 🚚: Current","New, 3rd Party FBM 🚚: Last visit","New, 3rd Party FBA: Current","New, 3rd Party FBM 🚚: 30 days avg.",Lowest FBM Seller,"New, 3rd Party FBM 🚚: 90 days avg.","New, 3rd Party FBA: 180 days avg.","New, 3rd Party FBM 🚚: Lowest","Count of retrieved live offers: New, FBM","New, 3rd Party FBM 🚚: 90 days drop %","New, 3rd Party FBA: Highest","New, 3rd Party FBA: 90 days drop %",FBA Pick&Pack Fee,Lowest FBA Seller,"New, 3rd Party FBA: 30 days avg.","New, 3rd Party FBA: 90 days avg.","New, 3rd Party FBM 🚚: 180 days avg.","New, 3rd Party FBA: Last visit","New, 3rd Party FBA: Lowest","New, 3rd Party FBA: 30 days drop %","New, 3rd Party FBM 🚚: 30 days drop %",List Price: Highest,List Price: Lowest,List Price: Current,List Price: 90 days avg.,List Price: 30 days avg.,List Price: 180 days avg.,List Price: 30 days drop %,"Count of retrieved live offers: New, FBA",Variation Count,ASIN,Title,URL: Keepa,Image Count,Product Codes: UPC,Product Group,Freq. Bought Together,Brand
0,26.20,79.99,Amazon offer is in stock and shippable,62.36,0.01,47.88,50.57,0.20,-0.06,NaN,49.76,47.13,62.24,49.13,79.99,0.00,26.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9512.0,9485.0,9527.0,0.0,9490.0,4.7,62.36,79.99,-5.0,49.70,yes,0.0,47.18,67.0,no,3.0,100.0,76.0,26.20,NaN,20.0,49.76,76.0,85.0,yes,153.0,98667.0,188.0,-75.0,140456.0,136403.0,238556.0,629351.0,300.0,4980.0,47.0,-142.0,77.00,77.00,NaN,NaN,77.00,A1BZ5E31ZCG12M,58.73,NaN,29.99,1.0,-31.0,NaN,NaN,12.25,-,NaN,NaN,55.67,NaN,NaN,NaN,0.0,79.99,79.99,79.99,79.99,79.99,79.99,0.0,1.0,2,B0882VT6G7,"TaoTronics HEPA Air Purifier for Home, Allerge...",https://keepa.com/#!product/1-B0882VT6G7,1,661094420206,Home,Unvailable,TaoTronics
1,NaN,NaN,no Amazon offer exists,NaN,1.00,NaN,NaN,0.00,0.01,NaN,29.45,29.60,29.41,29.57,31.00,0.00,29.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.80,31.99,-3.0,29.66,no,0.0,29.78,89.0,no,3.0,55.0,51.0,29.38,NaN,-3.0,30.72,49.0,56.0,no,31.0,157905.0,41.0,6.0,175148.0,163176.0,152902.0,334231.0,NaN,57399.0,10.0,3.0,31.00,29.45,NaN,NaN,29.41,A2LM7VTZ4M9M65,29.72,NaN,29.38,3.0,1.0,NaN,NaN,NaN,-,NaN,NaN,29.68,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,B0CGJV2Z68,Pepsi Soda 20oz Bottles (Pack of 10),https://keepa.com/#!product/1-B0CGJV2Z68,5,NaN,Grocery,Unvailable,B4Y
2,NaN,NaN,no Amazon offer exists,NaN,1.00,NaN,NaN,0.00,0.01,NaN,30.99,31.34,30.99,31.58,32.99,0.00,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-22.0,9.0,5.0,11.0,-37.0,8.0,4.9,31.4

In [5]:
print(df.columns.tolist())

['Amazon: Lowest', 'Amazon: Highest', 'Amazon: Availability of the Amazon offer', 'Amazon: 30 days avg.', 'Amazon: 90 days OOS', 'Amazon: 90 days avg.', 'Amazon: 180 days avg.', 'New: 30 days drop %', 'New: 90 days drop %', 'New: Last visit', 'New: Current', 'New: 90 days avg.', 'New: 30 days avg.', 'New: 180 days avg.', 'New: Highest', 'New: 90 days OOS', 'New: Lowest', 'eBay New 🚚: 90 days avg.', 'eBay New 🚚: Current', 'eBay New 🚚: 90 days drop %', 'eBay New 🚚: Highest', 'eBay New 🚚: 180 days avg.', 'eBay New 🚚: 30 days drop %', 'eBay New 🚚: 30 days avg.', 'eBay New 🚚: Lowest', 'Reviews: Review Count - 30 days drop %', 'Reviews: Review Count - 30 days avg.', 'Reviews: Review Count - 180 days avg.', 'Reviews: Review Count', 'Reviews: Review Count - 90 days drop %', 'Reviews: Review Count - 90 days avg.', 'Reviews: Rating', 'Buy Box 🚚: 30 days avg.', 'Buy Box 🚚: Highest', 'Buy Box 🚚: 90 days drop %', 'Buy Box 🚚: 180 days avg.', 'Buy Box: Is FBA', 'Buy Box 🚚: 90 days OOS', 'Buy Box 🚚: 9

In [6]:
if "Amazon: Availability of the Amazon offer" in df.columns:
    df.rename(columns={"Amazon: Availability of the Amazon offer": "amazon_offer"}, inplace=True)
else:
    raise KeyError("❌ 'Amazon: Availability of the Amazon offer' sütunu bulunamadı.")


In [7]:
df.Title.unique

<bound method Series.unique of 0       TaoTronics HEPA Air Purifier for Home, Allerge...
1                    Pepsi Soda 20oz Bottles (Pack of 10)
2                    Pepsi Soda 20oz Bottles (Pack of 12)
3       Pepsi Soda 20oz Bottles (Pack of 16, Total of ...
4       Clamato Original Tomato Cocktail, 5.5 fl oz ca...
                              ...                        
8302    Orient Therapy Lavender Gift Box with Air Fres...
8303    Orient Therapy Lavender Massage Oil – Infused ...
8304    Orient Therapy Odor Eliminator – Lavender Oil ...
8305    Orient Therapy Lavender Pet Shampoo for Dogs &...
8306    Orient Therapy Lavender Oil Reed Diffuser Set ...
Name: Title, Length: 8307, dtype: object>

In [8]:
df.Title.unique

<bound method Series.unique of 0       TaoTronics HEPA Air Purifier for Home, Allerge...
1                    Pepsi Soda 20oz Bottles (Pack of 10)
2                    Pepsi Soda 20oz Bottles (Pack of 12)
3       Pepsi Soda 20oz Bottles (Pack of 16, Total of ...
4       Clamato Original Tomato Cocktail, 5.5 fl oz ca...
                              ...                        
8302    Orient Therapy Lavender Gift Box with Air Fres...
8303    Orient Therapy Lavender Massage Oil – Infused ...
8304    Orient Therapy Odor Eliminator – Lavender Oil ...
8305    Orient Therapy Lavender Pet Shampoo for Dogs &...
8306    Orient Therapy Lavender Oil Reed Diffuser Set ...
Name: Title, Length: 8307, dtype: object>

In [9]:
df.info

<bound method DataFrame.info of       Amazon: Lowest  Amazon: Highest                            amazon_offer  \
0              26.20            79.99  Amazon offer is in stock and shippable   
1                NaN              NaN                  no Amazon offer exists   
2                NaN              NaN                  no Amazon offer exists   
3                NaN              NaN                  no Amazon offer exists   
4              11.62            42.04                  no Amazon offer exists   
...              ...              ...                                     ...   
8302             NaN              NaN                  no Amazon offer exists   
8303             NaN              NaN                  no Amazon offer exists   
8304             NaN              NaN                  no Amazon offer exists   
8305             NaN              NaN                  no Amazon offer exists   
8306             NaN              NaN                  no Amazon offer exists

In [10]:
 df = df.rename(columns={"URL: Keepa": "url_keepa"})

In [11]:
df.url_keepa.tail(20)

8287    https://keepa.com/#!product/1-B004DN2Z3A
8288    https://keepa.com/#!product/1-B00RZ75LE6
8289    https://keepa.com/#!product/1-B00B044ODO
8290    https://keepa.com/#!product/1-B0CLBC98YL
8291    https://keepa.com/#!product/1-B09VLJM7KH
8292    https://keepa.com/#!product/1-B0BMZNQKTM
8293    https://keepa.com/#!product/1-B09W14ZPN6
8294    https://keepa.com/#!product/1-B09V6RLFMV
8295    https://keepa.com/#!product/1-B09SNSBZ6X
8296    https://keepa.com/#!product/1-B0C2ZPFXSH
8297    https://keepa.com/#!product/1-B0C2JNTXK2
8298    https://keepa.com/#!product/1-B09W48X1Y9
8299    https://keepa.com/#!product/1-B09TG8D697
8300    https://keepa.com/#!product/1-B0BNZ3RRTJ
8301    https://keepa.com/#!product/1-B0BQCZDXLS
8302    https://keepa.com/#!product/1-B0C1P9TYG9
8303    https://keepa.com/#!product/1-B0BNZ4QNPP
8304    https://keepa.com/#!product/1-B0BNZH8C15
8305    https://keepa.com/#!product/1-B0BNZGXMCB
8306    https://keepa.com/#!product/1-B0BNYVCKRY
Name: url_keepa, dty

In [12]:
df.columns

Index(['Amazon: Lowest', 'Amazon: Highest', 'amazon_offer',
       'Amazon: 30 days avg.', 'Amazon: 90 days OOS', 'Amazon: 90 days avg.',
       'Amazon: 180 days avg.', 'New: 30 days drop %', 'New: 90 days drop %',
       'New: Last visit',
       ...
       'Count of retrieved live offers: New, FBA', 'Variation Count', 'ASIN',
       'Title', 'url_keepa', 'Image Count', 'Product Codes: UPC',
       'Product Group', 'Freq. Bought Together', 'Brand'],
      dtype='object', length=102)

In [13]:
import pandas as pd

# Sütun isimlerini temizle
df.columns = df.columns.str.strip()

# "URL: Keepa" sütununu kısalt
if "URL: Keepa" in df.columns:
    df.rename(columns={"URL: Keepa": "url_keepa"}, inplace=True)

# Amazon Availability sütununu bul ve amazon_offer olarak değiştir


# amazon_offer tam olarak "Amazon offer is in stock and shippable" olanları filtrele
stokta_olan = df[df["amazon_offer"] == "Amazon offer is in stock and shippable"]

# 'Sales Rank: 30 days avg.' sütunu varsa sırala, yoksa ilk 10 ürünü al
sira_sutun = "Sales Rank: 30 days avg."
if sira_sutun in stokta_olan.columns:
    trend_urunler = stokta_olan.sort_values(by=sira_sutun, ascending=False).head(10)
else:
    trend_urunler = stokta_olan.head(10)

# Brand yoksa Product Group kullan
if "Brand" not in trend_urunler.columns or trend_urunler["Brand"].isna().all():
    if "Product Group" in trend_urunler.columns:
        trend_urunler["Brand"] = trend_urunler["Product Group"].fillna("Bilinmiyor")
    else:
        trend_urunler["Brand"] = "Bilinmiyor"
else:
    trend_urunler["Brand"] = trend_urunler["Brand"].fillna(
        trend_urunler["Product Group"].fillna("Bilinmiyor") if "Product Group" in trend_urunler.columns else "Bilinmiyor"
    )

# Gösterilecek sütunlar
kolonlar = ["ASIN", "Brand", "Title", sira_sutun, "url_keepa"]
kolonlar = [k for k in kolonlar if k in trend_urunler.columns]

trend_urunler = trend_urunler[kolonlar]

# Excel çıktısı oluştur
trend_urunler.to_excel("trend_urunler.xlsx", index=False)
print("✅ Trend ürün listesi oluşturuldu: trend_urunler.xlsx")


✅ Trend ürün listesi oluşturuldu: trend_urunler.xlsx


In [32]:
df.columns = df.columns.str.strip()  # boşlukları temizle

filtrelenen_urunler = df[df['Title'].str.startswith('Orient Therapy Lavender Gift Box')]

# İster sadece belirli sütunları alabilirsiniz
sonuc = filtrelenen_urunler[['ASIN', 'Brand', 'Title', 'url_keepa', 'Sales Rank: 30 days avg.']]

# İlk 10 ürünü görmek için
print(sonuc.head(10))

            ASIN           Brand  \
3827  B0C1TCRZYH  ORIENT THERAPY   
3828  B0C1P8MZ8X  ORIENT THERAPY   
8302  B0C1P9TYG9  ORIENT THERAPY   

                                                  Title  \
3827  Orient Therapy Lavender Gift Box with Air Fres...   
3828  Orient Therapy Lavender Gift Box with Air Fres...   
8302  Orient Therapy Lavender Gift Box with Air Fres...   

                                     url_keepa  Sales Rank: 30 days avg.  
3827  https://keepa.com/#!product/1-B0C1TCRZYH                  792599.0  
3828  https://keepa.com/#!product/1-B0C1P8MZ8X                  809605.0  
8302  https://keepa.com/#!product/1-B0C1P9TYG9                       NaN  


In [14]:
import os
import pandas as pd

# (Opsiyonel) chardet ile otomatik encoding tespiti yapabilirsin.
try:
    import chardet
    HAVE_CHARDET = True
except Exception:
    HAVE_CHARDET = False

INPUT_PATH = r"C:\Users\beddi\Downloads\Internship\PazarTrendAsis\urun_data.csv"  # veya .xlsx
OUTPUT_PATH = r"C:\Users\beddi\Downloads\Internship\PazarTrendAsis\top10_sales.xlsx"

def detect_encoding(path, nbytes=100000):
    if not HAVE_CHARDET:
        return None
    with open(path, "rb") as f:
        raw = f.read(nbytes)
    res = chardet.detect(raw)
    return res.get("encoding")

def robust_read(path):
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xls", ".xlsx"]:
        print("📥 Excel dosyası algılandı. pandas.read_excel ile okunuyor...")
        return pd.read_excel(path)
    elif ext in [".csv", ".txt"]:
        print("📥 CSV dosyası algılandı. Encoding denemeleri başlatılıyor...")
        # Önce chardet ile dene (varsa)
        if HAVE_CHARDET:
            enc = detect_encoding(path)
            if enc:
                try:
                    print(f"  → chardet tahmini encoding: {enc} (deneniyor)")
                    return pd.read_csv(path, encoding=enc)
                except Exception as e:
                    print(f"  → chardet ile okuma başarısız: {e}")
        # Sırayla denenecek encodlar
        enc_list = ["utf-8-sig", "utf-8", "cp1254", "iso-8859-9", "latin1", "cp1252"]
        for enc in enc_list:
            try:
                print(f"  → {enc} ile deniyor...")
                return pd.read_csv(path, encoding=enc)
            except Exception as e:
                print(f"    - başarısız: {e}")
        # Son çare: errors='replace' ile oku
        try:
            print("  → Son çare: latin1 + errors='replace' ile okunuyor...")
            return pd.read_csv(path, encoding="latin1", errors="replace")
        except Exception as e:
            raise IOError(f"Dosya okunamadı. Son hata: {e}")
    else:
        raise ValueError("Desteklenmeyen dosya uzantısı: " + ext)

def main():
    if not os.path.exists(INPUT_PATH):
        raise FileNotFoundError(f"Girdi dosyası bulunamadı: {INPUT_PATH}")

    df = robust_read(INPUT_PATH)
    print("✅ Dosya başarıyla okundu. Satır sayısı:", len(df))

    # order_status filtresi: varsa küçük harfe çevirip uygun sipariş durumlarını kullan
    if "order_status" in df.columns:
        try:
            df = df[df["order_status"].astype(str).str.lower().isin(["shipped", "delivered", "completed", "fulfilled"])]
            print("🔎 order_status filtresi uygulandı.")
        except Exception:
            print("⚠ order_status filtresi uygulanamadı (sütun tipi sorunlu). Filtre atlandı.")

    # quantity ve item_price temizliği
    if "quantity" in df.columns:
        df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").fillna(0)
    else:
        raise KeyError("Dataframe'de 'quantity' sütunu yok.")

    if "item_price" in df.columns:
        df["item_price"] = pd.to_numeric(df["item_price"], errors="coerce")
    else:
        # varsa unit_price ile dene
        if "unit_price" in df.columns:
            df["item_price"] = pd.to_numeric(df["unit_price"], errors="coerce")
            print("ℹ 'item_price' yok, 'unit_price' kullanıldı.")
        else:
            print("⚠ 'item_price' veya 'unit_price' bulunamadı. avg_price NaN olabilir.")

    # Grupla ve top10
    group_cols = []
    if "product_name" in df.columns:
        group_cols.append("product_name")
    else:
        raise KeyError("Dataframe'de 'product_name' sütunu yok.")
    # ASIN veya SKU varsa ekle
    if "asin" in df.columns:
        group_cols.append("asin")
    elif "sku" in df.columns:
        group_cols.append("sku")
    else:
        print("ℹ 'asin' veya 'sku' bulunamadı; sadece product_name ile gruplanacak.")

    top_products = (
        df.groupby(group_cols, as_index=False)
          .agg(
              total_quantity = ("quantity", "sum"),
              avg_price = ("item_price", "mean"),
              sample_order_id = ("amazon_order_id", "first")
          )
          .sort_values("total_quantity", ascending=False)
          .head(10)
    )

    # Excel'e yaz (openpyxl kullanır)
    try:
        top_products.to_excel(OUTPUT_PATH, index=False, engine="openpyxl")
    except Exception as e:
        # fallback: to_excel default
        top_products.to_excel(OUTPUT_PATH, index=False)
    print(f"✅ Top 10 ürün '{OUTPUT_PATH}' dosyasına kaydedildi.")
    print(top_products)

if __name__ == "__main__":
    main()

📥 CSV dosyası algılandı. Encoding denemeleri başlatılıyor...
  → chardet tahmini encoding: utf-8 (deneniyor)
✅ Dosya başarıyla okundu. Satır sayısı: 102461
🔎 order_status filtresi uygulandı.
✅ Top 10 ürün 'C:\Users\beddi\Downloads\Internship\PazarTrendAsis\top10_sales.xlsx' dosyasına kaydedildi.
                                           product_name        asin  \
2069  JUMBO JUMBLE SNACK BOX (40 COUNT) PROTEIN GIFT...  B0BB47L81H   
2070  JUMBO JUMBLE SNACK BOX (45 COUNT) GIFT CARE PA...  B09ZZWYSDJ   
3271  PUFF BAR COLLAGEN PROTEIN MIXED BOX, BROWNIE B...  B0DKTT1C67   
1737  HARVEAST ORGANIC MEDJOOL DATES - CALIFORNIA GR...  B0CLBC4CCH   
1736  HARVEAST ORGANIC MEDJOOL DATES - CALIFORNIA GR...  B0CL5KYS9V   
3531  SEALY STERLING COLLECTION DOWN ALTERNATIVE PIL...  B0B3HB4TCN   
1744  HARVEAST SUN DRIED FIGS, 2 LBS - NATURAL TURKI...  B099ZY8LP1   
479   BIOXCIN HERBAL CARE SHAMPOO - BIOXCIN FORTE HE...  B0D8CDCMN5   
3575  SIETE FAMILY FOODS GRAIN FREE TORTILLA CHIPS V...  B0CY57G